# py_datatable Q & A Notebook

In [41]:
# Importing the necessary libraries
import datatable as dt
from datatable import f,by,count,sum,update,sort
dt.init_styles()
dt.options.display.head_nrows=4
dt.options.display.tail_nrows=4

## 1. Data Manipulations

### 1. 1 How to sort a datatable frame in descending order.?

We have created a sample dataframe with two columns such as product(character type) and totals(numeric type) using a frame object from dt and assigned it to a variable called X.

In [2]:
X = dt.Frame(product=["apples", "spam", "goo", "bobcat", "gold"], 
                 totals=[5.4, 2.777, 0.1, 2.9, 11.1])

In [3]:
X

,product,totals
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,apples,5.4
1,spam,2.777
2,goo,0.1
3,bobcat,2.9
4,gold,11.1


As you might have already know about the datatable syntax as below 



                                        DT[I,J,BY|SORT|JOIN]
                                        
                                       
                                       
                                       
                                       
For now look at the sort function, it takes eigther a single column or multiple columns in, and it would be applicable for character and numeric type fields


1. In below code chunk case-1 we have passed a column totals in sort so that it arranges the data frame in ascending order considering the column(total).

2. In code chunk case-2 the same column is given with appending a symbol(-) so that it arranges the data frame in descending order considering the column(total).

3. In code chunk case - 3 we are trying to arrange the dataframe in ascending order of the products column

In [4]:
# case - 1
X[:,:,sort(f.totals)]

,product,totals
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,goo,0.1
1,spam,2.777
2,bobcat,2.9
3,apples,5.4
4,gold,11.1


In [5]:
# case - 2 
X[:,:,sort(-f.totals)]

,product,totals
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,gold,11.1
1,apples,5.4
2,bobcat,2.9
3,spam,2.777
4,goo,0.1


In [6]:
# case - 3
X[:,:,sort(f.product)]

,product,totals
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,apples,5.4
1,bobcat,2.9
2,gold,11.1
3,goo,0.1
4,spam,2.777


Let us create a one more dataframe with repeated values of products as below

In [7]:
X = dt.Frame(products=['apples','spam','apples','gold','spam'],
             totals=[20,40,35,10,5])

We are now summing off the totals per each category of products and arrange it in descending order of newly created column tot_sum

In [8]:
X[:,{'tot_sum':sum(f.totals)},by(f.products)
 ][:,:,sort(-f.tot_sum)
  ]

,products,tot_sum
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,apples,55
1,spam,45
2,gold,10


### 1.2 How to count the number of instances for each category using group by in pydatadable?

Here is our basic syntax representation of datatable frame- 


                                        DT[I,J,BY|SORT|JOIN]

A sample dataframe created with the column name **languages** and would like to count how many of students are interested in learning each language category using aggregations such as by along with count,min,max,mean etc etc..

Yes, its correct we should use a function called **count** to caluclate the number of observations and let us see how it works below. 

In [9]:
prog_lang_dt = dt.Frame(languages= ['html', 'R', 'R', 'html', 'R', 'javascript',
                                    'R', 'javascript', 'html'])

In [10]:
prog_lang_dt

,languages
,▪▪▪▪
0,html
1,R
2,R
3,html
4,R
5,javascript
6,R
7,javascript
8,html


In [11]:
prog_lang_dt[:,count(),by(f.languages)]

,languages,count
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,R,4
1,html,3
2,javascript,2


If we would like like to rename a count column as total it can be done as follows,

In [12]:
prog_lang_dt[:,{'total':count()},by(f.languages)]

,languages,total
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,R,4
1,html,3
2,javascript,2


**count** can also take a column name as argument and report how many of non-missing entries in that specific column. for this example we will create a new dataframe as Y.

In [13]:
data = """
       id|charges|payment_method
       634-VHG|28|Cheque
       365-DQC|33.5|Credit card
       264-PPR|631|--
       845-AJO|42.3|
       789-KPO|56.9|Bank Transfer
       """

In [14]:
# read the data
Y = dt.fread(data, na_strings=['--', ''])

In [15]:
Y[:,count(f.payment_method)]

,payment_method
,▪▪▪▪▪▪▪▪
0,3


Here its simpy shows the count of payment methods are 3 and the remaining 2 observations are ignored

### 1.3 How to type cast a dataframe column in pydatatable?

We will create a dataframe with four columns such as cust_id,sales,profit_perc, and default

In [16]:
sales_DT = dt.Frame(

    {"cust_id":[893232.43],
     
     "sales":[1234532],
     
     "profit_perc":['10.43'],
     
     "default":[1]
    }
)

Check the each column datatype as below - 

In [17]:
sales_DT.stypes

(stype.float64, stype.int32, stype.str32, stype.int8)

Here are some key points:

-  cust_id is a type of float but in general customer id should be eigther integer or string type

-  sales is a type of int, it should not always be an integer and it may also be in float types

-  profict_perc is a type of string, here it should be a float type

-  default is a type of int and it should be a bool

**Note** : We have a syntax to be followed when we are to converting a column datatype from one to another as below

                           DT['Column_name']= new data type (int,floar,str,bool)

First, we will now try to apply the above formula on conveting a column type from float(**cust_id**) to integer 

In [18]:
sales_DT['cust_id'] = int

Here, let us verify the same whether it has become an integer type or not ?

In [19]:
sales_DT.stypes

(stype.int32, stype.int32, stype.str32, stype.int8)

Yes, it is converted. similarily we can convert a type from int(**sales**) to float and check.

In [20]:
sales_DT['sales'] = float

We have a column default with int type and we can have it as bool type.

In [21]:
sales_DT['default'] = bool

In [22]:
sales_DT.stypes

(stype.int32, stype.float64, stype.str32, stype.bool8)

So far we have seen the convertions from 

- int to float
- float to int 
- int to str 
- float to string 
- int to bool
- etc etc 

In [23]:
sales_DT

,cust_id,sales,profit_perc,default
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪
0,893232,1.23453e+06,10.43,1


That is OK, you have noticed or tried converting a column from string to any other types **(int,float,bool)**

In [24]:
sales_DT['profit_perc'] = float

NotImplementedError: Unable to cast `str32` into `float64`

**Note** : String to other type converions are not yet implemented in datatable versions till 0.10.1 and they would be surely implemented in the upcoming versions.

### 1.4 How to select columns based on their data types?

We will import a data from a specified URL source for this example as-

In [26]:
spotify_songs_dt = dt.fread('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')

Datafram dimensions(rows,columns) can be checked as below

In [27]:
spotify_songs_dt.shape

(32833, 23)

Spotify dataframe has come with about 32K observations and 23 columns and the columns types can be viewed as

In [28]:
spotify_songs_dt.stypes

(stype.str32,
 stype.str32,
 stype.str32,
 stype.int32,
 stype.str32,
 stype.str32,
 stype.str32,
 stype.str32,
 stype.str32,
 stype.str32,
 stype.str32,
 stype.float64,
 stype.float64,
 stype.int32,
 stype.float64,
 stype.bool8,
 stype.float64,
 stype.float64,
 stype.float64,
 stype.float64,
 stype.float64,
 stype.float64,
 stype.int32)

We would just like to take a look at the fields which are an any  type of ( int or string or bool or float ) etc etc. let us understand how it can be achived in py-datatable. we are already familiarized with datatable I,J sytax style as below. In J would always be made use to select the columns based  on given names or indices etc etc.. The required datatypes should be entered in the J expression so that it will display the respective observations.

                                DT[:,J]

Well, In a first attempt let us retrive the observations for all the interger columns as - 

In [42]:
spotify_songs_dt[:,dt.int32]

,track_popularity,key,duration_ms
,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,66,6,194754
1,67,11,162600
2,70,1,176616
3,60,7,169093
⋮,⋮,⋮,⋮
"32,829",20,0,353120
"32,830",14,6,210112
"32,831",15,2,367432
"32,832",27,5,337500


In a second attempt, we would like to see both float and bool type columns, and the below is the solution for it.

In [43]:
spotify_songs_dt[:,[dt.float64,dt.bool8]]

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,mode
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪
0,0.748,0.916,−2.634,0.0583,0.102,0,0.0653,0.518,122.036,1
1,0.726,0.815,−4.969,0.0373,0.0724,0.00421,0.357,0.693,99.972,1
2,0.675,0.931,−3.432,0.0742,0.0794,2.33e-05,0.11,0.613,124.008,0
3,0.718,0.93,−3.778,0.102,0.0287,9.43e-06,0.204,0.277,121.956,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
"32,829",0.522,0.786,−4.462,0.042,0.00171,0.00427,0.375,0.4,128.041,1
"32,830",0.529,0.821,−4.899,0.0481,0.108,1.11e-06,0.15,0.436,127.989,0
"32,831",0.626,0.888,−3.361,0.109,0.00792,0.127,0.343,0.308,128.008,1
"32,832",0.603,0.884,−4.571,0.0385,0.000133,0.341,0.742,0.0894,127.984,0


So here we have learnt that if more than one type of columns are required to be selected the types should be passed in J expression using a list i.e [dt.int32,dt.str32,dt.bool8] 

**Note** : In above two cases we have limited the output observations for 5 only. 

### 1.5 How do deselect columns from dataframe?

Deselecting the columns from dataframe is as important as selecting the columns. Deselection of columns can be done in J position itself specifying a function called **removed** along with the f expressions. let us look at the syntax first.

                DT[:,f[:].remove(cols to be kept a side)]

Here is our first example - deselect a column track_id from the spotify dataframe.

In [48]:
spotify_songs_dt[:,f[:].remove(f.track_id)]

,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,…,instrumentalness,liveness,valence,tempo,duration_ms
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,I Don't Care (with Justin Bieber) - Loud Luxury Re…,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury Rem…,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,…,0,0.0653,0.518,122.036,194754
1,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,…,0.00421,0.357,0.693,99.972,162600
2,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,…,2.33e-05,0.11,0.613,124.008,176616
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,…,9.43e-06,0.204,0.277,121.956,169093
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮
"32,829",Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20,6kD6KLxj7s8eCE3ABvAyf5,Closer Remixed,2013-03-08,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,…,0.00427,0.375,0.4,128.041,353120
"32,830",Sweet Surrender - Radio Edit,Starkillers,14,0ltWNSY9JgxoIZO4VzuCa6,Sweet Surrender (Radio Edit),2014-04-21,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,…,1.11e-06,0.15,0.436,127.989,210112
"32,831",Only For You - Maor Levi Remix,Mat Zo,15,1fGrOkHnHJcStl14zNx8Jy,Only For You (Remixes),2014-01-01,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,…,0.127,0.343,0.308,128.008,367432
"32,832",Typhoon - Original Mix,Julian Calor,27,0X3mUOm6MhxR7PzxG95rAo,Typhoon/Storm,2014-03-03,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,…,0.341,0.742,0.0894,127.984,337500


In a second case - deselect these four columns(track_album_id,track_album_name,playlist_name,playlist_genre) from spotify dataframe - 

In [47]:
spotify_songs_dt[:,f[:].remove([f.track_album_id,f.track_album_name,f.playlist_name,f.playlist_genre])]

,track_id,track_name,track_artist,track_popularity,track_album_release_date,playlist_id,playlist_subgenre,danceability,energy,key,…,instrumentalness,liveness,valence,tempo,duration_ms
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxury Re…,Ed Sheeran,66,2019-06-14,37i9dQZF1DXcZDD7cfEKhW,dance pop,0.748,0.916,6,…,0,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,2019-12-13,37i9dQZF1DXcZDD7cfEKhW,dance pop,0.726,0.815,11,…,0.00421,0.357,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,2019-07-05,37i9dQZF1DXcZDD7cfEKhW,dance pop,0.675,0.931,1,…,2.33e-05,0.11,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,2019-07-19,37i9dQZF1DXcZDD7cfEKhW,dance pop,0.718,0.93,7,…,9.43e-06,0.204,0.277,121.956,169093
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮
"32,829",5Aevni09Em4575077nkWHz,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20,2013-03-08,6jI1gFr6ANFtT8MmTvA2Ux,progressive electro house,0.522,0.786,0,…,0.00427,0.375,0.4,128.041,353120
"32,830",7ImMqPP3Q1yfUHvsdn7wEo,Sweet Surrender - Radio Edit,Starkillers,14,2014-04-21,6jI1gFr6ANFtT8MmTvA2Ux,progressive electro house,0.529,0.821,6,…,1.11e-06,0.15,0.436,127.989,210112
"32,831",2m69mhnfQ1Oq6lGtXuYhgX,Only For You - Maor Levi Remix,Mat Zo,15,2014-01-01,6jI1gFr6ANFtT8MmTvA2Ux,progressive electro house,0.626,0.888,2,…,0.127,0.343,0.308,128.008,367432
"32,832",29zWqhca3zt5NsckZqDf6c,Typhoon - Original Mix,Julian Calor,27,2014-03-03,6jI1gFr6ANFtT8MmTvA2Ux,progressive electro house,0.603,0.884,5,…,0.341,0.742,0.0894,127.984,337500


Here we have given a column names in a list and let us understand the anotomy of this syntax. 

- f[:]: it selects all the columns

- f[:].remove([f.x,f.y,f.z]) will deselect these three columns from others



What if our requirement is based on a type of column, of course it can also be achived with the same syntax as follows. and here we are deselecting all the string type columns from spotify dataframe.

In [51]:
spotify_songs_dt[:,f[:].remove(f[dt.str32])]

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,66,0.748,0.916,6,−2.634,1,0.0583,0.102,0,0.0653,0.518,122.036,194754
1,67,0.726,0.815,11,−4.969,1,0.0373,0.0724,0.00421,0.357,0.693,99.972,162600
2,70,0.675,0.931,1,−3.432,0,0.0742,0.0794,2.33e-05,0.11,0.613,124.008,176616
3,60,0.718,0.93,7,−3.778,1,0.102,0.0287,9.43e-06,0.204,0.277,121.956,169093
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
"32,829",20,0.522,0.786,0,−4.462,1,0.042,0.00171,0.00427,0.375,0.4,128.041,353120
"32,830",14,0.529,0.821,6,−4.899,0,0.0481,0.108,1.11e-06,0.15,0.436,127.989,210112
"32,831",15,0.626,0.888,2,−3.361,1,0.109,0.00792,0.127,0.343,0.308,128.008,367432
"32,832",27,0.603,0.884,5,−4.571,0,0.0385,0.000133,0.341,0.742,0.0894,127.984,337500
